# Thông tin nhóm

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# Câu hỏi

Cho các thông tin về căn nhà:

- Đường.
- Huyện/Quận.
- Mặt tiền (met).
- Đường vào (met).
- Hướng nhà.
- Chiều dài của nhà (met).
- Chiều rộng của nhà (met).
- Chính chủ (yes/no).
- Gần trung tâm TP, dễ dàng đi lại (yes/no).
- Loại nhà (nhà cấp 4, nhà lầu hay biệt thự?).
- Hẻm (yes/no).
- Nội thất (yes/no hay giá ước lượng bao nhiêu tiền?)
- Nhà mới xây (yes/no).
- Số lượng tầng.
- Số lượng phòng ngủ.
- Số lượng toliet.

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# Thu thập dữ liệu (parse HTML)

Dữ liệu thu thập từ trang web https://batdongsan.com.vn với bộ lọc là "Bán nhà riêng", "Khu vực TP.HCM". Thu được HTML cần parse: https://batdongsan.com.vn/ban-nha-rieng-tp-hcm

In [1]:
# Import các thư viện cần thiết
import urllib.robotparser
import json
import logging
import os.path
import time
import traceback
from datetime import datetime
from pprint import pprint

from requests_html import HTMLSession

In [2]:
# Tạo looger để ghi nhận lịch sử ở mỗi lần lấy dữ liệu
logger = logging.getLogger("crawler")
logFileName = f'{datetime.today().strftime("%Y-%m-%d_%H_%M_%S")}.log'
formatter = logging.Formatter("[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")
fhandler = logging.FileHandler(filename=logFileName)
fhandler.setLevel(logging.DEBUG)
fhandler.setFormatter(formatter)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.handlers = []
logger.addHandler(fhandler)
logger.addHandler(ch)
logger.setLevel(logging.DEBUG)


# Hàm trả về generator, generator náy trả về trang tiếp theo sau mỗi lần duyệt
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p{i}"
        i += 1


# Hàm parse để lấy thông tin chi tiết của căn nhà từ detail Url của tin rao bán nhà.      
def parseDetailPage(session: HTMLSession, url: str):
    global logger
    logger.info(f"Processing detail url: {url}")
    
    # Lấy mã HTML của trang web
    r = session.get(url)
    
    # Dictionary lưu lại kết quả
    result = {}
    
    # Lấy phần mô tả chi tiết của bản tin rao bán căn nhà
    result["description"] = r.html.find(".pm-desc", first=True).text

    # Lấy hết các thông tin nổi bật của nhà (ví dụ: địa chỉ, mặt tiền, số phòng ngủ, số tầng, nội thất,...)
    features = {}
    featuresTable = r.html.find(".table1", first=True) # các thông tin này nằm ở class table1 trong mã HTML
    rows = featuresTable.find(".row") # lấy hết tất cả các dòng của features Table
    for row in rows:
        key = row.find(".left", first=True).text
        value = row.find(".right", first=True).text
        features[key] = value
    result["features"] = features # Thêm phần feature vào phần kết quả muốn lấy

    return result


def parseSearchResultPage(fileName: str,
                          session: HTMLSession,
                          baseUrl: str = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm",
                          startPage: int = 1):
    global logger
    if os.path.exists(fileName):
        logger.error(f"File exists {fileName}")
        return

    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('https://batdongsan.com.vn/robots.txt')
    rp.read()

    try:
        with open(fileName, "w", encoding="utf8") as f:
            for url in urlGenerator(baseUrl, startPage):
                
                # Bỏ qua trang hiện tại nếu website không cho phép lấy.
                if not rp.can_fetch('*', url):
                    logger.info(f"SKIP {url}")
                    continue
                    
                logger.info(f"Processing url: {url}")
                
                # Lấy mã HTML của trang web
                r = session.get(url)
                
                # Tìm phần tử "not found" của trang kết quả tìm kiếm,
                # nếu phần tử này tồn tại tức là trang này không có kết quả nào
                # -> đã lấy hết
                notFound = r.html.find(
                    "#LeftMainContent__productSearchResult_pnlNotFound", first=True
                )
                if notFound is not None:
                    logger.info(f"{url} Not found")
                    break

                # Lấy hết các bản tin rao bán nhà
                items = r.html.find(".search-productItem")
                for item in items:
                    result = {} # Dictionary này chứa tất cả các thông tin của căn nhà cần rao bán
                    
                    # Lấy title
                    titleAnchor = item.find(".p-title a", first=True)
                    result["title"] = titleAnchor.text
                    
                    # Lấy detail Url
                    result["detailUrl"] = titleAnchor.absolute_links.pop()
                    
                    # Lấy giá nhà
                    result["price"] = item.find(".product-price", first=True).text
                    
                    # Lấy diện tích (met vuông)
                    result["area"] = item.find(".product-area", first=True).text
                    
                    # Lấy khu vực (huyện/quận)
                    result["district"] = item.find(
                        ".product-city-dist", first=True
                    ).text
                    
                    # Lấy thời gian đăng tin
                    result["uptime"] = item.find(".uptime", first=True).text
                    
                    # Kiểm tra xem có được phép lấy dữ liệu từ trang detail hay không?
                    if rp.can_fetch('*', result["detailUrl"]):
                        # Lấy thông tin mô tả chi tiết của căn nhà
                        result["detail"] = parseDetailPage(session, result["detailUrl"])
                    else:
                        logger.info(f"SKIP {result['detailUrl']}")
                    
                    # Ghi xuống file
                    json.dump(result, f, ensure_ascii=False)
                    f.write("\n")

                # sleep 2s khi lấy xong mỗi trang (chứa 20 tin đăng bán nhà)
                time.sleep(2)
                
    except Exception as e:
        logger.error(f"{e}\n{traceback.format_exc}")

In [ ]:
session = HTMLSession()

# Tạo tên file lưu lại kết quả đã parse dữ liệu từ trang web, tên file chính là thời gian tiến hành parse
resultFileName = f'data/{datetime.today().strftime("%Y-%m-%d_%H_%M_%S")}.json'

# Parse trang kết quả tìm kiếm
parseSearchResultPage(resultFileName, session)

[2019-11-30 13:21:54,431][crawler][INFO] - Processing url: https://batdongsan.com.vn/ban-nha-rieng-tp-hcm
[2019-11-30 13:21:56,750][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-23-2-phuong-binh-tri-dong-b/can-khu-ten-lua-90m2-3-lau-co-ham-de-xe-chinh-chu-can-ban-pr23679945
[2019-11-30 13:21:57,520][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-doan-nguyen-tuan-1-xa-hung-long-5/binh-chanh-1ty3-gan-cho-0934117173-pr22722319
[2019-11-30 13:21:58,389][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-dong-hung-thuan-6-phuong-tan-hung-thuan-1/can-gap-5x10m2-cong-n-50m2-mot-tret-1-lung-3-lau-pr23671000
[2019-11-30 13:21:59,364][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/ban-nha-rieng-duong-lien-khu-4-5-phuong-binh-hung-hoa-b/133-16-4-5-quan-tan-tp-hcm-pr22894731
[2019-11-30 13:22:00,346][crawler][INFO] - Processing detail url: https://batdongsan.com.vn/b

# Tiền xử lý dữ liệu